In [ ]:
import json
import re
import os
import requests
import time
import itertools
from tqdm import tqdm
from bs4 import BeautifulSoup
from requests_html import HTMLSession
from concurrent.futures import ThreadPoolExecutor
session = HTMLSession()

In [ ]:
def write_to_json(lst, fn):
    import json
    with open(fn, "a") as file:
        for item in lst:
            x = json.dumps(item, ensure_ascii=False)
            file.write(x + "\n")

# get link

In [ ]:
max_workers = 20

def get_links(page):
    url = f"https://youbaby.my/blog/page/{page}/"

    r = session.get(url)
    soup = BeautifulSoup(r.content, "lxml")

    articles = soup.find_all("h3", {"class": "elementor-post__title"})

    for item in articles:
        data = {
            "url": item.find('a').get('href'),
            "page": url
            }
        write_to_json([data], f"output/youbabymy-links.txt")

pages = list(range(1, 10))
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(get_links, x) for x in pages]
    
    for future in tqdm(futures, total=len(pages)):
        result = future.result()

# get articles

In [ ]:
max_workers = 50

with open(f"output/youbabymy-links.txt") as f:
    lst = f.readlines()

def get_data(url):
    r = session.get(url)
    soup = BeautifulSoup(r.content, "lxml")

    basic = soup.find("ul", {"class": "elementor-inline-items elementor-icon-list-items elementor-post-info"})
    published_date = re.sub('\s+', ' ', basic.find("li", {"itemprop": "datePublished"}).text).strip()
    published_time = re.sub('\s+', ' ', basic.find("span", {"class": "elementor-icon-list-text elementor-post-info__item elementor-post-info__item--type-time"}).text).strip()
    comment_counts = re.sub('\s+', ' ', basic.find("li", {"itemprop": "commentCount"}).text).strip()

    content = soup.find("div", {"class": "elementor-element elementor-element-7f9c0fa8 elementor-widget elementor-widget-theme-post-content"})
    
    try:
        content.find('div', {'class': 'ez-toc-title-container'}).decompose()
    except:
        pass

    data = {
        "url": url,
        "published_date": published_date,
        "published_time": published_time,
        "comment_count": comment_counts,
        "contents": content.get_text().strip().replace('\n', '')
    }
    
    write_to_json([data], f"output/youbabymy-data.json")
    
    content = None
    
    
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(get_data, json.loads(x)['url']) for x in lst]
    
    for future in tqdm(futures, total=len(lst)):
        result = future.result()
